In [ ]:
!pip uninstall python-snappy

In [ ]:
!pip install apache_beam

In [ ]:
!pip install apache-beam[gcp]

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

### What resource we need to Train this large data??: GPU

### Completed Tasks:

--> GEE Asset/Crop polygons

--> polygon area estimation for each crop type data

--> Plante/NICFI Pixel count for each crops, each polygons

--> Graphic representation for area vs Pixel count

### Next tasks:

--> Cloud masking for Planet (Algorthms)

--> Image Normalization: Scale the pixel values (e.g., 0-255) to a range that is typical for neural network inputs, such as 0-1 or -1 to 1.

--> Data Augmentation: To increase the diversity of the training data and prevent overfitting, apply transformations like rotations, translations, scaling, and horizontal flipping.

--> Patch Extraction: For high-resolution images, it might be necessary to create smaller, manageable patches. This makes the training process more efficient and helps in handling large images during deployment.

--> DL (FCNN) Trainings
--> Parallel ML training

### Reading corner about crop mapping in Senegal
SEN4STAT/ESA: https://www.esa-sen4stat.org/user-stories/senegal-prototype/

EOSTAT/FAO: https://data.apps.fao.org/catalog/dataset/5c377b2b-3c2e-4b70-afd7-0c80900b68bb/resource/50bc9ff5-95d2-40cd-af12-6aee2cfcc4ae

In [ ]:
#Lib imports:
import ee
#print('Using EE version ', ee.__version__)
import folium
#print('Using Folium version ', folium.__version__)
from os import MFD_HUGE_1MB
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import layers
#print('Using TF version ', tf.__version__)
from typing import Dict, Iterable, List, Tuple
#from google.colab import auth

In [ ]:
# authenticate with user credentials
#auth.authenticate_user()

In [ ]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the Earth Engine object with Google Cloud project ID
project_id = 'ee-kkidia3'
ee.Initialize(project=project_id)


In [ ]:
# Initialize the GEE module.
ee.Initialize()

In [ ]:
# @title Default title text

def list_ee_assets(folder_path):
    """ List all assets in the specified Earth Engine folder path """
    try:
        assets_list = ee.data.getList({'id': folder_path})
        for asset in assets_list:
            print(asset['id'])
    except Exception as e:
        print("Error accessing Earth Engine assets:", e)

# GEE Asset path
#folder_path = 'users/kkidia3'

# Call the function to list assets
#list_ee_assets(folder_path)


### Crop fields polygon for rainfed season 2018-2020

In [ ]:
#Data base from 2018-2020: Crop field plot polygon.In the dataset different crop types assembeled in one file.
data_2018 = ee.FeatureCollection("projects/ee-djitastar/assets/data_2018")
data_2019 = ee.FeatureCollection("projects/ee-djitastar/assets/data_2019")
data_2020 = ee.FeatureCollection("projects/ee-djitastar/assets/data_2020")

#merge for the rainfed croping season 2018-2020.
merge18_20 = data_2018.merge(data_2019).merge(data_2020)

attribute table of data

In [ ]:
from shapely.geometry import shape
import geopandas as gpd

# Function to convert Earth Engine FeatureCollection to a pandas DataFrame
def fc_to_df(fc):
    features = fc.getInfo()['features']
    dict_list = [f['properties'] for f in features]
    for d, f in zip(dict_list, features):
        d['geometry'] = shape(f['geometry'])
    return pd.DataFrame(dict_list)

# Convert the entire feature collection to a pandas DataFrame
df = fc_to_df(data_2018)
print(df.head())

# to convert this DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(gdf.head())

# save the DataFrame to a CSV file if needed
#df.to_csv('data_2018_features.csv', index=False)
#print("Feature table saved to data_2018_features.csv")


### Visualize 18-20

In [ ]:
# @title Default title text
# Define a function to add Earth Engine vector data as a layer to a Folium map
def add_ee_vector_layer(feature_collection, style, layer_name, map_object):
    painted = ee.Image().paint(feature_collection, 'constant', 2)  # Here 'constant' is a dummy property for visualization
    map_id_dict = painted.getMapId(style)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=layer_name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Create a Folium map object
center = [14.4974, -14.4524]  # Center of the map (e.g., Senegal)
m1 = folium.Map(location=center, zoom_start=7)

# Styling for the vector layer
style = {
    'color': 'blue',  # Line color
    'fillColor': '00000000',  # Fill color with opacity (00)
}

# Add the merged crop fields to the map
add_ee_vector_layer(merge18_20, style, 'Merged Crops 2019-2020', m1)

# Add a layer control panel to the map
folium.LayerControl().add_to(m1)

# Display the map
m1


In [ ]:
print(data_2018.getInfo())

### Polygons for FY2023

In [ ]:
# Function to load individual crop feature collections
def load_crop_data(asset_id):
    return ee.FeatureCollection(asset_id)

# Load individual rainfed 2023 crop feature collections
gnut = load_crop_data('projects/ee-kkidia3/assets/groundnut')
gsorrel = load_crop_data('projects/ee-kkidia3/assets/guinea_sorrel')
rice = load_crop_data('projects/ee-kkidia3/assets/rice')
sesame = load_crop_data('projects/ee-kkidia3/assets/sesame')
soye = load_crop_data('projects/ee-kkidia3/assets/soye')
watermelon = load_crop_data('projects/ee-kkidia3/assets/watermelon')
cassava = load_crop_data('projects/ee-kkidia3/assets/cassava')
gnutmix = load_crop_data('projects/ee-kkidia3/assets/groundnut_mixed')
milletmix = load_crop_data('projects/ee-kkidia3/assets/millet_mixed')
okra = load_crop_data('projects/ee-kkidia3/assets/okra')
sorgh = load_crop_data('projects/ee-kkidia3/assets/sorgh')
wheat = load_crop_data('projects/ee-kkidia3/assets/wheat')
millet = load_crop_data('projects/ee-kkidia3/assets/millet')
cowpmix = load_crop_data('projects/ee-kkidia3/assets/cowpea_mixed')
fallow = load_crop_data('projects/ee-kkidia3/assets/fellow')

# Merge all crop fields into a single feature collection
merged2023 = gnut.merge(gsorrel).merge(rice).merge(sesame).merge(soye).merge(watermelon)\
                .merge(cassava).merge(gnutmix).merge(milletmix).merge(okra).merge(sorgh)\
                .merge(wheat).merge(millet).merge(cowpmix).merge(fallow)



In [ ]:
print(gnut.getInfo())

Select sample 2 polygons from goundnet polygons

In [ ]:
#print(poly_2)
# @title Default title text
# Select two specific polygons by their indices or properties. e.g gnut
# Here, I want to select the first two polygons.
poly_2 = gnut.toList(2)
# Get the individual polygons
polygon1 = ee.Feature(poly_2.get(0))
polygon2 = ee.Feature(poly_2.get(1))

# Print the geometries of the selected polygons to verify.
print('Polygon 1_Blue:', polygon1.geometry().getInfo())
print('Polygon 2_Red:', polygon2.geometry().getInfo())
# Create a map centered at an arbitrary point @polygon1
map_center = [13.237198228125724,-14.715474917616827]  #set this to a blue selected @polygon1
m = folium.Map(location=map_center, zoom_start=30) #

# Define a function to add a feature to the folium map.
def add_feature_to_map(feature, map_obj, color):
    geom = feature.geometry().getInfo()
    coords = geom['coordinates']
    if geom['type'] == 'Polygon':
        folium.Polygon(locations=[(pt[1], pt[0]) for pt in coords[0]], color=color).add_to(map_obj)
    elif geom['type'] == 'MultiPolygon':
        for poly in coords:
            folium.Polygon(locations=[(pt[1], pt[0]) for pt in poly[0]], color=color).add_to(map_obj)

# Add polygons to the map
add_feature_to_map(polygon1, m, 'blue')
add_feature_to_map(polygon2, m, 'red')

# Display the map
#m


In [ ]:
# Define the time range
START = '2023-09-01'
END = '2023-09-30'

# Polygon coordinates
COORDS = [[-14.715474917616827, 13.237198228125724],
          [-14.714971029320179, 13.237131335847982],
          [-14.714841720208264, 13.237135818148086],
          [-14.714864057837882, 13.237282954362941],
          [-14.714921993298699, 13.237389931092336],
          [-14.714756997435309, 13.237452415652427],
          [-14.714890756204218, 13.23747022665853],
          [-14.71482392130252, 13.237563840257419],
          [-14.714792684227906, 13.237653054642704],
          [-14.714663375265559, 13.23760402506512],
          [-14.714667824987172, 13.237737716223846],
          [-14.714556315002342, 13.237751104481083],
          [-14.714435994662244, 13.23802313618169],
          [-14.714774885327822, 13.238223778014577],
          [-14.714971029320179, 13.238299595698706],
          [-14.715474917616827, 13.237198228125724]] #this coordinate is based on polygon1 (groundnut)

# Convert coordinates to ee.Geometry
TEST_ROI = ee.Geometry.Polygon([COORDS])

PATCH_SIZE = 128  # Pixels
SCALE = 10        # Meters per pixel for Sentinel
SCALE2 = 4.47  #Planet

# Pre-compute a geographic coordinate system.
proj = ee.Projection('EPSG:4326').atScale(SCALE).getInfo()

# Get scales out of the transform.
SCALE_X = proj['transform'][0]
SCALE_Y = -proj['transform'][4]

PATCH_SIZE = 128
OFFSET_X = -SCALE_X * PATCH_SIZE / 2
OFFSET_Y = -SCALE_Y * PATCH_SIZE / 2

REQUEST = {
    'fileFormat': 'NPY',
    'grid': {
        'dimensions': {
            'width': PATCH_SIZE,
            'height': PATCH_SIZE
        },
        'affineTransform': {
            'scaleX': SCALE_X,
            'shearX': 0,
            'shearY': 0,
            'scaleY': SCALE_Y,
            'translateX': OFFSET_X,
            'translateY': OFFSET_Y
        },
        'crsCode': proj['crs']
    }
}

VALIDATION_RATIO = 0.2
POINTS_PER_CLASS = 64
TEST_POINT = ee.Geometry.Point(COORDS[0])

# The TEST_ROI is precisely 16 times the size of an individual patch.
TEST_ROI = TEST_POINT.buffer(2 * PATCH_SIZE * SCALE, maxError=1).bounds(maxError=1)

# A list of regions from which to sample training data.
REGIONS = merged2023
# Predictors.
INPUT_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'] #sentinel
INPUT_BANDS2 = ['R', 'G', 'B'] #planet
# Target variable.
OUTPUT_BANDS = ['landcover']
# Input stack.
BANDS = INPUT_BANDS + OUTPUT_BANDS
# Map from input name to a fixed length feature.
FEATURES_DICT = {
    k: tf.io.FixedLenFeature(shape=[PATCH_SIZE, PATCH_SIZE], dtype=tf.float32)
    for k in BANDS
}

CLASSIFICATIONS = {
    "💧 Water":              "419BDF",
    "🌳 Trees":              "397D49",
    "🌾 Grass":              "88B053",
    "🌿 Flooded vegetation": "7A87C6",
    "🚜 Crops":              "E49635",
    "🪴 Shrub and scrub":    "DFC35A",
    "🏗️ Built-up areas":     "C4281B",
    "🪨 Bare ground":        "A59B8F",
    "❄️ Snow and ice":       "B39FE1",
}


### Classifieres for crop type

In [ ]:
import matplotlib.patches as mpatches
# Define a color map for each crop type
color_map = {
    'groundnut': '#FF6347',  # Tomato red
    'guinea_sorrel': '#FFD700',  # Gold
    'rice': '#32CD32',  # Lime Green
    'sesame': '#8A2BE2',  # Blue Violet
    'soye': '#FF4500',  # Orange Red
    'watermelon': '#00FA9A',  # Medium Spring Green
    'cassava': '#7B68EE',  # Medium Slate Blue
    'groundnut_mixed': '#D2691E',  # Chocolate
    'millet_mixed': '#DAA520',  # Golden Rod
    'okra': '#ADFF2F',  # Green Yellow
    'sorgh': '#FF00FF',  # Magenta
    'wheat': '#F4A460',  # Sandy Brown
    'millet': '#00CED1',  # Dark Turquoise
    'cowpea_mixed': '#FF1493',  # Deep Pink
    'fallow': '#708090'   # Slate Gray
}

# Create legend patches
legend_patches = [mpatches.Patch(color=color, label=key) for key, color in color_map.items()]

# Create the legend
plt.figure(figsize=(5, 5))
plt.legend(handles=legend_patches, loc='center', title="Crop Legend", ncol=2)
plt.axis('off')  # Hide axes
plt.show()


for Worldcover and dynmaic world

In [ ]:
def display_legend():
  reset_color = "\u001b[0m"
  colored = lambda red, green, blue: f"\033[48;2;{red};{green};{blue}m"
  for name, color in CLASSIFICATIONS.items():
    red   = int(color[0:2], 16)
    green = int(color[2:4], 16)
    blue  = int(color[4:6], 16)
    print(f"{colored(red, green, blue)}   {reset_color} {name}")

display_legend()

Compare the location based on DYNAMICWORLD

In [ ]:
# @title Default title text
image = (
    ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
    .filterDate(START, END)
    .median()
)

vis_params = {
  'max': len(CLASSIFICATIONS) - 1,
  'palette': list(CLASSIFICATIONS.values()),
  'bands': ['label'],
}
folium.Map(
    location=(COORDS[1], COORDS[0]),
    zoom_start=10,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

### Sentinel-2 surface reflectance

We will use a cloud-freee composite of Sentinel-2 surface reflectance data for predictors.  See [the Code Editor example](https://code.earthengine.google.com/?scriptPath=Examples%3ACloud%20Masking%2FSentinel2%20Cloud%20And%20Shadow) for details.

SENTINEL

In [ ]:
def get_s2_composite(roi, start, end):
  s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
  s2sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  s2c = s2c.filterBounds(roi.buffer(100*1000, 1000)).filterDate(start, end)
  s2sr = s2sr.filterBounds(roi.buffer(100*1000, 1000)).filterDate(start, end)

  def index_join(collection_a, collection_b, property_name):
    joined = ee.ImageCollection(
        ee.Join.saveFirst(property_name).apply(
            primary=collection_a,
            secondary=collection_b,
            condition=ee.Filter.equals(
                leftField='system:index',
                rightField='system:index')))
    return joined.map(
        lambda image: image.addBands(ee.Image(image.get(property_name))))

  def mask_image(image):
    prob = image.select('probability')
    return image.select('B.*').divide(10000).updateMask(prob.lt(50))

  with_cloud_probability = index_join(s2sr, s2c, 'cloud_probability')
  masked = ee.ImageCollection(with_cloud_probability.map(mask_image))
  return masked.select(INPUT_BANDS).median().float().unmask(0)

image = get_s2_composite(TEST_POINT, START, END)

vis_params = {
  'min': 0,
  'max': 0.4,
  'bands': ['B4', 'B3', 'B2'],
}

folium.Map(
    location=(13.237198228125724, -14.715474917616827), #Senegal
    zoom_start=10,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)



NICFI/PLANET

In [ ]:

# Define the region of interest (ROI)
TEST_POINT = ee.Geometry.Point([-14.715474917616827, 13.237198228125724])

def get_planet_composite(roi, start, end):
    # Adjust the dataset references for Planet/NICFI
    planet_ic = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

    # Filter by date and region
    planet_ic = planet_ic.filterBounds(roi.buffer(100*1000, 1000)).filterDate(start, end)

    # Planet NICFI doesn't have cloud probability bands, we will assume all images are usable
    def mask_image(image):
        return image.select(['R', 'G', 'B']).divide(10000)  # Adjust bands as needed

    masked = planet_ic.map(mask_image)
    return masked.median().float().unmask(0)

image = get_planet_composite(TEST_POINT, START, END)

# Visualization parameters for Planet NICFI
vis_params = {
    'min': 0,
    'max': 0.4,
    'bands': ['G', 'R', 'B'],  # Adjust the bands as needed
}

# Define a method for displaying Earth Engine image tiles with Folium.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map and add the image layer
map = folium.Map(
    location=[13.237198228125724, -14.715474917616827],  # Senegal
    zoom_start=10,
)

# Add the Planet NICFI layer to the map
map.add_ee_layer(image, vis_params, 'Planet NICFI Composite')

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Display the map
map




Compare the sample location to ESA worldcover. This is noting to do with the training

In [ ]:
def landcover_image() -> ee.Image:
    # Remap the ESA classifications into the Dynamic World classifications
    fromValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
    toValues = [1, 5, 2, 4, 6, 7, 8, 0, 3, 3, 7]
    return (
        ee.ImageCollection('ESA/WorldCover/v200')
        .first()
        .select('Map')
        .remap(fromValues, toValues)
        .rename(OUTPUT_BANDS)
    )

image = landcover_image()

vis_params = {
  'bands': OUTPUT_BANDS,
  'palette': list(color_map.values()), #CLASSIFICATION lenged as an option
}

folium.Map(
    location=(13.237198228125724, -14.715474917616827),
    zoom_start=10,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

In [ ]:
from google.api_core import exceptions, retry
import io
import numpy as np
import matplotlib.pyplot as plt
import requests
from typing import Dict, Iterable, List, Tuple


def sample_random_points(region: ee.Geometry, points_per_class: int) -> Iterable[List]:
  """Get a generator of random points in the region."""
  image = landcover_image().select(OUTPUT_BANDS).int()
  points = image.stratifiedSample(
      points_per_class,
      region=region,
      scale=SCALE,
      geometries=True,
  )
  for point in points.toList(points.size()).getInfo():
      yield point['geometry']['coordinates']


@retry.Retry()
def get_training_patch(coords: List[float]) -> np.ndarray:
  """Get a training patch centered on the coordinates."""
  point = ee.Geometry.Point(coords)
  image = get_s2_composite(point, START, END).addBands(landcover_image())
  request = dict(REQUEST)
  request['expression'] = image
  request['grid']['affineTransform']['translateX'] = coords[0] + OFFSET_X
  request['grid']['affineTransform']['translateY'] = coords[1] + OFFSET_Y
  return np.load(io.BytesIO(ee.data.computePixels(request)))


In [ ]:
sample = sample_random_points(TEST_ROI, POINTS_PER_CLASS)
coords = next(sample)
print(f"coords: {coords}")

patch = get_training_patch(coords)
print(f"patch shape={patch.shape} bands={len(patch.dtype)}")
print(f"dtype: {patch.dtype}")

Render the test patch to check the predictors
NOTE: This is the fake legend the the training legend should be based on crop types class

In [ ]:
def render_rgb_images(values: np.ndarray, min=0.0, max=1.0) -> np.ndarray:
  scaled_values = (values - min) / (max - min)
  rgb_values = scaled_values * 255
  return rgb_values.astype(np.uint8)

def render_sentinel2(patch: np.ndarray, min=0.0, max=0.3) -> np.ndarray:
  red   = patch["B4"]
  green = patch["B3"]
  blue  = patch["B2"]
  rgb_patch = np.stack([red, green, blue], axis=-1)
  return render_rgb_images(rgb_patch, min, max)

plt.imshow(render_sentinel2(patch))

In [ ]:
print(patch)

In [ ]:
def render_classifications(values: np.ndarray, palette: List[str]) -> np.ndarray:
  # Create a color map from a hex color palette.
  xs = np.linspace(0, len(palette), 256)
  indices = np.arange(len(palette))
  color_map = np.array([
        np.interp(xs, indices, [int(c[0:2], 16) for c in palette]),  # red
        np.interp(xs, indices, [int(c[2:4], 16) for c in palette]),  # green
        np.interp(xs, indices, [int(c[4:6], 16) for c in palette]),  # blue
  ]).astype(np.uint8).transpose()

  color_indices = (values / len(palette) * 255).astype(np.uint8)
  return np.take(color_map, color_indices, axis=0)

def render_landcover(patch: np.ndarray) -> np.ndarray:
  return render_classifications(patch, list(CLASSIFICATIONS.values()))

landcover_rgb = render_landcover(patch["landcover"])

print(f"landcover_rgb: {landcover_rgb.dtype} {landcover_rgb.shape}")
plt.imshow(landcover_rgb)
display_legend()

In [ ]:
def serialize(patch: np.ndarray) -> bytes:
  """Serializes a patch of data into a tf.train.Example proto."""
  features = {
      name: tf.train.Feature(
          float_list=tf.train.FloatList(value=patch[name].flatten())
      )
      for name in patch.dtype.names
  }
  example = tf.train.Example(features=tf.train.Features(feature=features))
  return example.SerializeToString()

serialized = serialize(patch)
print(f"serialized: {type(serialized).__name__} ({len(serialized)})")

In [ ]:
%%time

def split_dataset(element, num_partitions: int) -> int:
  weights = [1 - VALIDATION_RATIO, VALIDATION_RATIO]
  return random.choices([0, 1], weights)[0]

beam_options = PipelineOptions(
    [], direct_num_workers=len(REGIONS), direct_running_mode='multi_threading')

with beam.Pipeline(options=beam_options) as pipeline:
  training_data, validation_data = (
      pipeline
      | "Create regions" >> beam.Create(REGIONS)
      | "Sample random points" >> beam.FlatMap(sample_random_points, POINTS_PER_CLASS)
      | "Get patch" >> beam.Map(get_training_patch)
      | "Serialize" >> beam.Map(serialize)
      | "Split dataset" >> beam.Partition(split_dataset, 2)
  )

  training_data | "Write training data" >> beam.io.WriteToTFRecord(
      f"gs://{BUCKET}/crop-2023-datasets/training", file_name_suffix=".tfrecord.gz"
  )
  validation_data | "Write validation data" >> beam.io.WriteToTFRecord(
      f"gs://{BUCKET}/crop-2023-datasets/validation", file_name_suffix=".tfrecord.gz"
  )

### Visulaize the polygons in map

In [ ]:
# @title Default title text
# Define a function to add Earth Engine vector data as a layer to a Folium map
def add_ee_vector_layer(feature_collection, style, layer_name, map_object):
    painted = ee.Image().paint(feature_collection, 'constant', 2)  # Here 'constant' is a dummy property for visualization
    map_id_dict = painted.getMapId(style)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=layer_name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Create a Folium map object
center = [14.4974, -14.4524]  # Center of the map in  Senegal
m = folium.Map(location=center, zoom_start=7)

# Styling for the vector layer
style = {
    'color': 'blue',  # Line color
    'fillColor': '00000000',  # Fill color with opacity (00)
}

# Add the merged crop fields to the map
add_ee_vector_layer(merged2023, style, 'Merged Crops 2023', m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

# Display the map
#m

Remove overlapping polygons from your merged feature collection. Overlaps is due to data collection

In [ ]:
# Identify and remove overlapping polygons
def remove_overlaps(feature_collection):
    non_overlapping = ee.FeatureCollection([])
    while True:
        # Find the first feature
        first = feature_collection.first()
        if first is None:
            break
        first = ee.Feature(first)

        # Add the first feature to the non-overlapping collection
        non_overlapping = non_overlapping.merge(ee.FeatureCollection([first]))

        # Remove features that overlap with the first feature
        feature_collection = feature_collection.filter(ee.Filter.disjoint('.geo', first.geometry()))

    return non_overlapping

# Apply the function to remove overlaps
merged2023_non_overlapping = remove_overlaps(merged2023)

# Print the result (optional, for verification purposes)
print(merged2023_non_overlapping.size().getInfo())

For example let me sort out only two polygons out of maney gnut polygons for closer look and simple analysis.

In [ ]:
# @title Default title text
# Select two specific polygons by their indices or properties. e.g gnut
# Here, I want to select the first two polygons.
poly_2 = gnut.toList(2)

#print(poly_2)

# Get the individual polygons
polygon1 = ee.Feature(poly_2.get(0))
polygon2 = ee.Feature(poly_2.get(1))

# Print the geometries of the selected polygons to verify.
print('Polygon 1_Blue:', polygon1.geometry().getInfo())
print('Polygon 2_Red:', polygon2.geometry().getInfo())


# Create a map centered at an arbitrary point @polygon1
map_center = [13.237198228125724,-14.715474917616827]  #set this to a blue selected @polygon1
m = folium.Map(location=map_center, zoom_start=30) #

# Define a function to add a feature to the folium map.
def add_feature_to_map(feature, map_obj, color):
    geom = feature.geometry().getInfo()
    coords = geom['coordinates']
    if geom['type'] == 'Polygon':
        folium.Polygon(locations=[(pt[1], pt[0]) for pt in coords[0]], color=color).add_to(map_obj)
    elif geom['type'] == 'MultiPolygon':
        for poly in coords:
            folium.Polygon(locations=[(pt[1], pt[0]) for pt in poly[0]], color=color).add_to(map_obj)

# Add polygons to the map
add_feature_to_map(polygon1, m, 'blue')
add_feature_to_map(polygon2, m, 'red')

# Display the map
m

In [ ]:
# Calculate the total area of polygons in square meters for Rainfed season 2023. Example groundnut
area = gnut.geometry().area().format('%.1f').getInfo()
print(f"Area M2: {area} square meters OR,")

# Calculate the area in hectares (1 hectare = 10,000 square meters)
area_ha = gnut.geometry().area().divide(10000).format('%.1f').getInfo() # Convert square meters to hectares
print(f"Area Ha: {area_ha} hectares")

# Calculate the perimeter in meters
perimeter = gnut.geometry().perimeter().format('%.1f').getInfo()
print(f"Perimeter: {perimeter} meters")

# Get the centroid of the polygon
centroid = gnut.geometry().centroid().getInfo()
print(f"Centroid: {centroid}")

# Get the bounding box as a GeoJSON
bounding_box = gnut.geometry().bounds().getInfo()
print(f"Bounding Box: {bounding_box}")



### Estimate the each polygon area (m2), for each crop e.g Groundnt


In [ ]:
# Define a function to calculate the area of each polygon in hectares
#def calc_area(feature):
    #return feature.set('area_ha', ee.Number(feature.geometry().area().divide(10000)).format('%.3f'))

# Define a function to calculate the area of each polygon in m2
def calc_area(feature):
  return feature.set('area_m2', ee.Number(feature.geometry().area())
  .format('%.1f'))#.divide(10000)) & # .format('%.1f') use to limit the digits

# Apply the function to each feature in the collection
area_mapped = merged2023.map(calc_area)

# Fetch the mapped area information from the server
areas_info = area_mapped.limit(10).getInfo()  # Limiting to the first 10 features directly to show everthing remove .limit(10)

# Iterate through the first 10 features and print area in hectares with 3 decimal places
for feature in areas_info['features']:
    area = feature['properties']['area_m2'] ##m2
    print(f"Gnut Polygon_Area: {area} m2") #m2


### Area (ha) distributions e.g Groundnut

In [ ]:
# Define a function to calculate the area of each polygon in hectares
#def calc_area(feature):
    #return feature.set('area_ha', feature.geometry().area().divide(10000))  # Convert from square meters to hectares

# Define a function to calculate the area of each polygon in hectares
def calc_area(feature):
    return feature.set('area_ha', feature.geometry().area().divide(10000)) #--> # Convert from square meters to hectares

# Apply the function to each feature in the collection
area_mapped = gnut.map(calc_area)

# Extract and print the area of each polygon
areas_info = area_mapped.getInfo()

# Extracting areas into a list for plotting
areas = [feature['properties']['area_ha'] for feature in areas_info['features']]

# Plotting the distribution of polygon areas
plt.figure(figsize=(10, 6))
plt.hist(areas, bins='auto', color='skyblue', alpha=0.8, rwidth=0.85)
plt.title('Distribution of Polygon Areas in Ha for Groundnut')
plt.xlabel('Area (Ha)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Show area of each crop

In [ ]:
#  'gnut', 'gsorrel', 'rice', etc. are already defined as ee.Geometry() or ee.Feature() objects representing each crop
crop_types = {
    'Gnut': gnut,
    'G. Sorrel': gsorrel,
    'Rice': rice,
    'Sesame': sesame,
    'Soye': soye,
    'Watermelon': watermelon,
    'Cassava': cassava,
    'Gnut Mix': gnutmix,
    'Millet Mix': milletmix,
    'Okra': okra,
    'Sorgh': sorgh,
    'Wheat': wheat,
    'Millet': millet,
    'Cowpea Mix': cowpmix,
    'Fallow': fallow
}

# Calculate area for each crop and store in a dictionary
#areas = {name: crop.geometry().area().divide(10000).getInfo() for name, crop in crop_types.items()} #In Hectares
areas = {name: crop.geometry().area().getInfo() for name, crop in crop_types.items()}


# Create a DataFrame from the area dictionary
area_df = pd.DataFrame(list(areas.items()), columns=['Crop Type', 'Area (Squere meteres M2)'])

# Display the DataFrame
print(area_df)



Graphic of each crop in area coverages

In [ ]:
area_df = pd.DataFrame(list(areas.items()), columns=['Crop Type', 'Area (M2)'])

# Sort the DataFrame by area for better visualization
area_df.sort_values('Area (M2)', ascending=False, inplace=True)

# Plotting
plt.figure(figsize=(12, 8))
plt.bar(area_df['Crop Type'], area_df['Area (M2)'], color='skyblue')
plt.xlabel('Crop Type')
plt.ylabel('Area in M2')
plt.title('Area of Each Crop Type for FY2023')
plt.xticks(rotation=90)  # Rotate crop type names for better readability
plt.grid(False)
plt.show()

### Count Pixels (NICFI) for each crop field represnetation of each specteral bands e.g Groundnut

In [ ]:
# time range
start_date = '2023-09-01'
end_date = '2023-09-30'

# Load the Planet/NICFI imagery
imagery = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa').filterDate(start_date, end_date).filterBounds(merged2023)  # Assuming 'gnut' is the ee.Geometry() of my area of interest
#bands info
bands = ee.ImageCollection(imagery).first()
print(bands.bandNames().getInfo())

#################  Planet Pixel count ####### Actual pixel count from PLANET

# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Mask the image with the polygon
    masked_image = image.clip(gnut) #remember 1 polygone selected for closer see demo

    # Count pixels - 4.77 X 4.77 m (~5X5m) actual Planet/NICFI pixel resolution
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=4.77,  # Scale in meters; 5m resolution take time to estimate (timeout)
        # to avoid time out estimation use 10X10m and convert it in to 5X5 by devide the result 4
        maxPixels=1e9  # Adjust maxPixels if needed to handle large areas
    )

    # Need to return an ee.Feature for the .map() function
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])



In [ ]:
# Define the time range
start_date = '2023-09-01'
end_date = '2023-09-30'


# Load the Planet/NICFI imagery
imagery = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
            .filterDate(start_date, end_date) \
            .filterBounds(merged2023)

# Print band information
bands = imagery.first()
print(bands.bandNames().getInfo())

# Function to mask and count pixels within the AOI
def count_pixels(image):
    # Mask the image with the polygon
    masked_image = image.clip(merged2023)

    # Count pixels - 10m resolution for faster computation
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=merged2023,
        scale=10,  # Scale in meters; coarser resolution for faster computation
        maxPixels=1e6  # Adjust maxPixels if needed to handle large areas
    )

    # Need to return an ee.Feature for the .map() function
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

### pixel esimates for each crop considering area not pixels

In [ ]:
# Assuming these are the individual crop polygons. others to be added from other crop years.
crop_types = {
    'gnut': gnut, 'gsorrel': gsorrel, 'rice': rice, 'sesame': sesame,
    'soye': soye, 'watermelon': watermelon, 'cassava': cassava,
    'gnutmix': gnutmix, 'milletmix': milletmix, 'okra': okra,
    'sorgh': sorgh, 'wheat': wheat, 'millet': millet, 'cowpmix': cowpmix,
    'fallow': fallow
}

def count_pixels(crop_name, crop_polygon):
    # Filter the imagery to the bounds of the crop polygon
    crop_imagery = imagery.filterBounds(crop_polygon)

    # Apply the pixel counting for each image in the filtered collection
    def pixel_count(image):
        masked_image = image.clip(crop_polygon)
        pixel_count = masked_image.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=crop_polygon,
            scale=10,  # Adjust the scale based on the actual resolution of NICFI imagery
            maxPixels=1e9
        )
        return ee.Feature(None, pixel_count)

    pixel_counts = crop_imagery.map(pixel_count)
    pixel_counts_info = pixel_counts.getInfo()

    # Print results for each crop type
    print(crop_name)
    for feature in pixel_counts_info['features']:
        print(feature['properties'])

# Run the pixel counting for each crop type
for crop_name, crop_polygon in crop_types.items():
    count_pixels(crop_name, crop_polygon)

### Pixel Exclusion representes less (e.g 40%) with in inside the polygon area.

Masking Pixels: Pixels with less than 40% coverage by the polygon are excluded using the mask method, where mask = fraction.gte(0.4) creates a mask that includes only pixels where the fraction is 40% or more.

- Band Selection: When creating the mask, the code now ensures that a single band is selected using image.select(0). This selects the first band of the image, assuming the first band is suitable for your masking purposes. Adjust the band selection if necessary based on the bands available in your specific imagery.

- Mask Application: By using single_band_image.mask() in the fraction calculation, we ensure that any operations involving masks deal with a single-band image, thus avoiding band mismatch errors.

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Ensure that we work with a single band for mask creation
    # Here, you could potentially select a specific band or reduce to a single band
    # As an example, if the image has multiple bands, use just one (e.g., the first band) for mask calculations
    single_band_image = image.select(1)

    # Calculate the fraction of the pixel area that overlaps with the polygon using a constant image
    constantImage = ee.Image.constant(1).clip(gnut)
    fraction = constantImage.updateMask(single_band_image.mask()).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gnut,
        scale=5,  # estimation with planet 5x5 is very difficult
        maxPixels=1e9
    ).get('constant')

    # Threshold the fraction to include only pixels with >= 40% coverage
    mask = ee.Image(fraction).gte(0.4)

    # Mask the image with the calculated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

only one polygon from gnut only for learning use

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Ensure that we work with a single band for mask creation
    single_band_image = image.select(1)  # Select the first band (assuming 1-based index)

    # Calculate the fraction of the pixel area that overlaps with the polygon using a constant image
    constant_image = ee.Image.constant(1).clip(polygon1)
    fraction = constant_image.updateMask(single_band_image.mask()).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gnut.geometry(),
        scale=5,  # estimation with planet 5x5 is very difficult
        maxPixels=1e9
    ).get('constant')

    # Threshold the fraction to include only pixels with >= 40% coverage
    mask = ee.Image.constant(fraction).gte(0.4)

    # Mask the image with the calculated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut.geometry(),
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

In [ ]:
# Function to mask and count pixels within the 'gnut' polygon
def count_pixels(image):
    # Create a single band composite by averaging RGB and NIR bands
    composite = image.expression(
        "(R + G + B + N) / 4",  # Expression combining the bands
        {
            'R': image.select('R'),  #  'R' is the Red band
            'G': image.select('G'),  # 'G' is the Green band
            'B': image.select('B'),  #  'B' is the Blue band
            'N': image.select('N')   #  'N' is the NIR band
        }
    )

    # Generate a mask based on the composite value, adjusting threshold as necessary
    mask = composite.gt(0.2)  # Example threshold

    # Mask the image with the generated mask
    masked_image = image.updateMask(mask)

    # Count pixels
    pixel_count = masked_image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=gnut,
        scale=5,
        maxPixels=1e9
    )

    # Return an ee.Feature for the .map() function to work properly
    return ee.Feature(None, pixel_count)

# Apply the pixel counting to each image in the collection
pixel_counts = imagery.map(count_pixels)

# Get information from the ImageCollection
pixel_counts_info = pixel_counts.getInfo()

# Print out the results
for feature in pixel_counts_info['features']:
    print(feature['properties'])

Pixel counts using area coverage regardless of the specteral bands (NICFI + 4.77m X 4.77 M resolutions)

In [ ]:
resolution = 4.77  # Resolution of NICFI imagery in meters (4.77m x 4.77m per pixel)


# Function to calculate pixel count
def calculate_pixels(crop):
    area = crop.geometry().area()  # Area in square meters
    pixel_area = resolution * resolution  # Area of one pixel in square meters
    pixel_count = area.divide(pixel_area)  # Number of pixels
    return pixel_count.getInfo()

# Calculate pixel count for each crop and store in a dictionary
pixel_counts = {name: calculate_pixels(crop) for name, crop in crop_types.items()}

# Create a DataFrame from the area and pixel count dictionaries
area_df = pd.DataFrame(list(pixel_counts.items()), columns=['Crop Type', 'Pixel Count'])

# Sort the DataFrame by 'Pixel Count' in descending order
area_df = area_df.sort_values(by='Pixel Count', ascending=False)
# Display the DataFrame
print(area_df)

In [ ]:
# Plotting
#plt.figure(figsize=(10, 6))
#plt.bar(area_df['Crop Type'], area_df['Pixel Count'], color='green')
#plt.xlabel('Crop Type')
#plt.ylabel('Pixel Count')
#plt.title('Pixel Count by Crop Type (Ascending Order)')
#plt.xticks(rotation=45, ha='right')
#plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
#plt.show()
####################
# Plotting using seaborn lib.
plt.figure(figsize=(12, 8))
sns.barplot(x='Crop Type', y='Pixel Count', data=area_df, palette='viridis')  # Using 'viridis' palette for varying colors
plt.xlabel('Crop Type')
plt.ylabel('Pixel Count')
plt.title('Pixel Count by Crop Type - for data base 2(FY2023)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
plt.show()

Area vs Pixel count

In [ ]:
# Assuming 'crop_types' dictionary is predefined with ee.Geometry() objects
resolution = 4.77  # Resolution of NICFI imagery in meters

# Function to calculate pixel count
def calculate_pixels(area):
    pixel_area = resolution * resolution  # Area of one pixel in square meters
    return area / pixel_area

# Calculate area and pixel count for each crop and store in dictionaries
area_and_pixels = []
for name, crop in crop_types.items():
    area = crop.geometry().area().getInfo()  # Area in square meters
    pixel_count = calculate_pixels(area)
    area_and_pixels.append((name, area, pixel_count))

# Create a DataFrame from the results
df = pd.DataFrame(area_and_pixels, columns=['Crop Type', 'Area (Square Meters)', 'Pixel Count'])

# Sort the DataFrame by 'Pixel Count' in descending order (optional)
df.sort_values(by='Pixel Count', ascending=False, inplace=True)

# Plotting
fig, ax1 = plt.subplots(figsize=(14, 8))

# Bar plot for Area using Seaborn
color = 'tab:red'
sns.barplot(x='Crop Type', y='Area (Square Meters)', data=df, palette='viridis', ax=ax1)
ax1.set_xlabel('Crop Type')
ax1.set_ylabel('Area (Square Meters)', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a twin Axes sharing the x-axis for Pixel Count
ax2 = ax1.twinx()
color = 'tab:blue'
sns.lineplot(x='Crop Type', y='Pixel Count', data=df, sort=False, marker='o', color='red', ax=ax2)
ax2.set_ylabel('Pixel Count', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Improve layout and set x-axis labels rotation
plt.xticks(rotation=90)
fig.tight_layout()

plt.show()

### Garba

Hi Garba, please use the above functions and generate the mean/median pixel specteral band values for "R", "G", "B" and "N" values of each pixel for each crop. Let me know if you need my assitance or have questions on the above functions. Perhaps you can show it in vector/tabular form.

--------------------------------
R      |    G  |     B  |     N
-------------------------------
       |       |        |  

**Steps**

To enhance the quality and usability of the images for further analysis or applications Normalization and calibration of images are essential processes in image processing and computer vision.  

**1. Data Collection**: Obtain multispectral images of the crops that include the required spectral bands (R, G, B, and NIR).

**2. Image Preprocessing**:

2.1. Image Normalization

In [ ]:


# Load the image
image = cv2.imread('/path/to/your/image.jpg')

# Convert BGR image to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Normalize the image to range [0, 1]
normalized_image = image / 255.0

# Alternatively, you can normalize to range [-1, 1]
# normalized_image = (image / 127.5) - 1.0

# Display the original and normalized images
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image)

plt.subplot(1, 2, 2)
plt.title('Normalized Image')
plt.imshow(normalized_image)

plt.show()


2.2. Calibration

In [ ]:
import cv2
import numpy as np
import glob

# Termination criteria for corner sub-pixel accuracy
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# Arrays to store object points and image points from all images
objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane

# Load calibration images
images = glob.glob('/path/to/calibration/images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)

    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save the calibration results
np.savez('calibration_data.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

# Undistort an image using the calibration results
img = cv2.imread('/path/to/your/test/image.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# Crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

# Display the result
cv2.imshow('calibrated', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()


2.3. Alignment

  **  Install OpenCV**: Install OpenCV in your Colab environment to use its functionalities.

    **Load and Display Images**: Load the two images you want to align. Display them using Matplotlib to ensure they are loaded correctly.

    **Detect ORB Features and Compute Descriptors**: Use the ORB detector to find keypoints and compute descriptors for both images.

    **Match Features Using BFMatcher**: Match the descriptors using BFMatcher, and sort the matches based on their distance (quality).

    **Find Homography and Warp Image**: Extract the coordinates of the matched points, compute the homography matrix using RANSAC, and apply the homography to warp the second image to align it with the first image.

1. **Install OpenCV**: First, ensure you have OpenCV installed in your Colab environment.

In [ ]:
!pip install opencv-python-headless


**2. Load and Display Images**: Load the images you want to align.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the images in grayscale
img1 = cv2.imread('/path/to/your/image1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/path/to/your/image2.jpg', cv2.IMREAD_GRAYSCALE)

# Display the images
plt.subplot(1, 2, 1)
plt.title('Image 1')
plt.imshow(img1, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Image 2')
plt.imshow(img2, cmap='gray')

plt.show()


**3. Detect ORB Features and Compute Descriptor**s:

In [ ]:
# Initialize the ORB detector
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)


**4. Match Features Using BFMatcher**:

In [ ]:
# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors1, descriptors2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw top matches
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
plt.figure(figsize=(20, 10))
plt.imshow(img_matches)
plt.title('Feature Matches')
plt.show()


**5. Find Homography and Warp Image:**

In [ ]:
# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography matrix
h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)

# Use homography to warp the image
height, width = img1.shape
aligned_img = cv2.warpPerspective(img2, h, (width, height))

# Display the aligned image
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.title('Reference Image')
plt.imshow(img1, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Aligned Image')
plt.imshow(aligned_img, cmap='gray')

plt.show()


**Extract Pixel Values: Extract the pixel values for each spectral band (R, G, B, NIR) for each segmented crop area.**

Clipping

3. **Segmentation**

**4. Extract Pixel Values**: Extract the pixel values for each spectral band (R, G, B, NIR) for each segmented crop area.

**5. Compute Statistics:**

    Mean: Calculate the mean value for each spectral band across all pixels for each crop.
    Median: Calculate the median value for each spectral band across all pixels for each crop.

### Cyrille
